In [ ]:
# ROBERT HEETER
# ELEC 378 Machine Learning
# 3 March 2023

# PROBLEM SET 7


In [ ]:
# PROBLEM 3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

# PART A

df_train = pd.read_csv('train.csv')
train_nums = df_train.select_dtypes(include='number') # select only numerical data types

data = train_nums.values
X = data[:,~np.isnan(data).any(axis=0)]
y = X[:,-1] # final column of the data set is the actual sale price
X = np.delete(X,-1,1)

X = X[:,1:] # remove first column of data set since sklearn centers the data with fit_intercept

print(f'rank(X) = {np.linalg.matrix_rank(X)}')
print(f'\ncondition(X) = {np.linalg.cond(X)}')
print(f'\ndiagonal matrix of singular values of X from SVD:\n{np.linalg.svd(X)[1]}')


In [ ]:
# PART B

# ridge regression
errors = []
alphas = np.arange(1,10000,10)
for a in alphas:
    clf = Ridge(alpha=a, fit_intercept=True).fit(X, y)
    y_approx = clf.predict(X)
    errors.append(np.mean((np.abs(y-y_approx))/y)*100)
plt.figure(0)
plt.plot(alphas,errors)
plt.xlabel('alpha')
plt.ylabel('error (%)')
plt.title('Error of ridge regression predicted sale price')
plt.show()
print(f'\nminimum ridge regression error:\nerror (%) = {errors[(np.argsort(errors))[0]]}\nalpha = {alphas[(np.argsort(errors))[0]]}')
print(f'\nridge regression predicted sale prices ($):\n{y_approx}')


# unregularized linear regression (from HW #6)
X_m = np.hstack((np.ones([len(X),1]),X)) # append column of 1's to data matrix to account for intercept (center of data)
psuedo_X = np.linalg.pinv(X_m) # compute Moore-Penrose pseudoinverse, pseudo_X = (X^T*X)^-1 * X^T
w = np.matmul(psuedo_X, y) # find optimal weightings, w = pseudo_X*y
y_approx = np.matmul(X_m, w) # find calculated y (sale price) from regression, y_approx = X*w (+ error)
error = np.mean((np.abs(y-y_approx))/y)*100
print(f'\nunregularized linear regression error = {error}%')
print(f'\nunregularized linear regression predicted sale prices ($):\n{y_approx}')


In [ ]:
# PART C

# lasso regression
errors = []
alphas = np.arange(1,10000,10)
for a in alphas:
    clf = Lasso(alpha=a, fit_intercept=True).fit(X, y)
    y_approx = clf.predict(X)
    errors.append(np.mean((np.abs(y-y_approx))/y)*100)
plt.figure(1)
plt.plot(alphas,errors)
plt.xlabel('alpha')
plt.ylabel('error (%)')
plt.title('Error of lasso regression predicted sale price')
plt.show()
print(f'\nminimum lasso regression error:\nerror (%) = {errors[(np.argsort(errors))[0]]}\nalpha = {alphas[(np.argsort(errors))[0]]}')
print(f'\nlasso regression predicted sale prices ($):\n{y_approx}')

# unregularized linear regression (from HW #6)
X_m = np.hstack((np.ones([len(X),1]),X)) # append column of 1's to data matrix to account for intercept (center of data)
psuedo_X = np.linalg.pinv(X_m) # compute Moore-Penrose pseudoinverse, pseudo_X = (X^T*X)^-1 * X^T
w = np.matmul(psuedo_X, y) # find optimal weightings, w = pseudo_X*y
y_approx = np.matmul(X_m, w) # find calculated y (sale price) from regression, y_approx = X*w (+ error)
error = np.mean((np.abs(y-y_approx))/y)*100
print(f'\nunregularized linear regression error = {error}%')
print(f'\nunregularized linear regression predicted sale prices ($):\n{y_approx}')


In [ ]:
# PART D

# assess feature weights between methodologies

clf = Ridge(alpha=642, fit_intercept=True).fit(X, y) # ideal ridge regression based on part B (alpha = 642)
w_ridge = clf.coef_
print(f'\nridge regression feature weights (w):\n{w_ridge}')

clf = Lasso(alpha=1268, fit_intercept=True).fit(X, y) # ideal lasso regression based on part C (alpha = 1268)
w_lasso = clf.coef_
print(f'\nlasso regression feature weights (w):\n{w_lasso}')

# unregularized linear regression (from HW #6)
X_m = np.hstack((np.ones([len(X),1]),X)) # append column of 1's to data matrix to account for intercept (center of data)
psuedo_X = np.linalg.pinv(X_m) # compute Moore-Penrose pseudoinverse, pseudo_X = (X^T*X)^-1 * X^T
w_unreg = np.matmul(psuedo_X, y) # find optimal weightings, w = pseudo_X*y
print(f'\nunregularized linear regression feature weights (w):\n{w_unreg}')


# assess feature importance ranking between methodologies

labels = train_nums.columns[~np.isnan(data).any(axis=0)] # remove all columns that were ignored for linear regression
labels_ridge = labels[1:-1] # remove first and last column of indices from labels
w_i_ridge = np.flip(np.argsort(abs(w_ridge)))
print('\nmost to least impactful features (ridge regression):')
print(labels_ridge[w_i_ridge])

labels_lasso = labels[1:-1] # remove first and last column of indices from labels
w_i_lasso = np.flip(np.argsort(abs(w_lasso)))
print('\nmost to least impactful features (lasso regression):')
print(labels_lasso[w_i_lasso])

labels_unreg = labels[1:-1] # remove first and last column of indices from labels
w_i_unreg = np.flip(np.argsort(abs(w_unreg[1:]))) # ignore intercept (column 0) for weightings
print('\nmost to least impactful features (unregularized linear regression):')
print(labels_unreg[w_i_unreg])


In [ ]:
# PROBLEM 4

import numpy as np
from matplotlib import pyplot as plt

from sklearn.datasets import load_digits
from scipy.spatial.distance import cdist
from scipy.stats import mode
from sklearn.model_selection import train_test_split

# PART A

# load digits
digits = load_digits()

# split digits into training and test data
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=0)

# example image
index = 100
plt.imshow(X_train[index].reshape((8,8)))
plt.show()
print(f'Digit: {y_train[index]}')


In [ ]:
def knn(X_fit, y_fit, X_predict, n_neighbors=5, metric='euclidean'):
    '''
    inputs:
        X_fit - 2D array containing all training data points
        y_fit - 2D array containing all training data labels
        X_predict - 2D array containing all data points to classify
        n_neighbors - K
        metric - see scipy.spatial.distance.cdist:
                 https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html 
                 
    returns: a 1D array of predicted labels for X_predict.
    '''

    # ensure that X_predict is two dimensional; this only matters if X_predict is one data point
    if X_predict.ndim < 2:
        X_predict = [X_predict]

    # calculate distances
    distances = cdist(X_predict, X_fit, metric)
    
    # find the data indices of least distance for each point; keep the closest n_neighbors data points for each point
    closest = np.argsort(distances,axis=1)[:,0:n_neighbors]
    
    # find the label of the n_neighbors closest data points
    closest_labels = y_fit[closest]
    
    # get the mode of each row and return the resulting 1D array of labels
    y_predict = mode(closest_labels, axis=1, keepdims=True)[0][:,0]
    
    return y_predict


In [ ]:
# PART B,C,D

metrics = ['euclidean', 'cityblock', 'chebyshev']
plt.figure(2)

for metric in metrics:
    print(f'\nDISTANCE METRIC = {metric}')
    
    errors = []
    Ks = range(1,200)
    
    for K in Ks:

        predictions = knn(X_fit=X_train, y_fit=y_train, X_predict=X_test, n_neighbors=K, metric=metric)
        error = np.sum((y_test != predictions)*1)/len(predictions)*100
        
        errors.append(error)
        
        if K <= 20:
            print(f'K = {K}; misclassification error: {error}%')
    
    plt.plot(Ks,errors)

plt.legend(metrics)
plt.xlabel('K (number of neighbors)')
plt.ylabel('misclassification error (%)')
plt.title('Error of K-nearest neighbors digit classification')
plt.show()
